In [ ]:
# ============================================================
# Shadow Dexterous Hand + Crosley Alarm Clock
# 5 grasp trials, fixed camera, no debug prints
# ============================================================

import os
import time
import mujoco
import mujoco.viewer
import gymnasium_robotics
import numpy as np

# -------------------------------
# CONFIG
# -------------------------------


ROOT_DIR = r"C:\Users\rad\itmo\new_roms"
XML_PATH = os.path.join(ROOT_DIR, "project", "models", "hand", "hand_manipulate_clock.xml")


# ASSETS_DIR = os.path.join(
#     os.path.dirname(gymnasium_robotics.__file__),
#     "envs", "assets", "hand"
# )

# XML_PATH = os.path.join(
#     ASSETS_DIR,
#     "hand_manipulate_clock.xml"
# )

NUM_TRIALS  = 5
OPEN_STEPS  = 40
CLOSE_STEPS = 120
HOLD_STEPS  = 200

# print("Assets dir:", ASSETS_DIR)
# print("Clock scene XML exists:", os.path.exists(XML_PATH))

print("XML exists:", os.path.exists(XML_PATH))
print("XML:", XML_PATH)

# -------------------------------
# LOAD MUJOCO MODEL
# -------------------------------

model = mujoco.MjModel.from_xml_path(XML_PATH)
data  = mujoco.MjData(model)

print("\nScene loaded successfully")
print("Bodies:", model.nbody)
print("Joints:", model.njnt)
print("Actuators:", model.nu)

# -------------------------------
#ТУТ СЕНСОРЫ ПРОВЕРЯЮТСЯ НА НАЛИЧИЕ
touch_sensors = [
    model.sensor(i).name
    for i in range(model.nsensor)
    if model.sensor(i).type == mujoco.mjtSensor.mjSENS_TOUCH
]

print("Touch sensors:", touch_sensors)
# -------------------------------

print("\n=== ACTUATORS ===")
for i in range(model.nu):
    print(i, model.actuator(i).name)

# пальцы: 2..19 (0–1 — запястье)
FINGER_ACTUATORS = list(range(2, 20))

# -------------------------------
# HELPERS
# -------------------------------

def get_contacts(model, data, object_keyword="clock"):
    contacts = []
    for i in range(data.ncon):
        c = data.contact[i]
        g1 = model.geom(c.geom1).name
        g2 = model.geom(c.geom2).name
        if object_keyword in g1 or object_keyword in g2:
            contacts.append((g1, g2))
    return contacts


def is_grasping(model, data, min_contacts=2):
    return len(get_contacts(model, data)) >= min_contacts


def reset_simulation(model, data, viewer):
    mujoco.mj_resetData(model, data)
    mujoco.mj_forward(model, data)
    viewer.sync()
    time.sleep(0.2)

# -------------------------------
# LAUNCH VIEWER
# -------------------------------

viewer = mujoco.viewer.launch_passive(model, data)

# -------------------------------
# GRASP TRIALS
# -------------------------------

for trial in range(NUM_TRIALS):
    print(f"\n==============================")
    print(f" TRIAL {trial + 1}/{NUM_TRIALS}")
    print(f"==============================")

    # --- RESET ---
    reset_simulation(model, data, viewer)

    # --- OPEN HAND ---
    for _ in range(OPEN_STEPS):
        data.ctrl[:] = 0.0
        mujoco.mj_step(model, data)
        viewer.sync()
        time.sleep(0.005)

    # --- CLOSE FINGERS (FAST) ---
    for _ in range(CLOSE_STEPS):
        data.ctrl[:] = 0.0
        for idx in FINGER_ACTUATORS:
            data.ctrl[idx] = 1.0
        mujoco.mj_step(model, data)
        viewer.sync()
        time.sleep(0.005)

    print("Contacts after close:", get_contacts(model, data))
    print("Is grasping:", is_grasping(model, data))

    # --- HOLD ---
    for _ in range(HOLD_STEPS):
        for idx in FINGER_ACTUATORS:
            data.ctrl[idx] = 1.0
        mujoco.mj_step(model, data)
        viewer.sync()
        time.sleep(0.01)

    print("Final grasp check:", is_grasping(model, data))

# -------------------------------
# KEEP WINDOW OPEN
# -------------------------------

print("\nAll trials finished.")
print("MuJoCo window will stay open.")
print("Close the viewer window manually to exit.")

while viewer.is_running():
    mujoco.mj_step(model, data)
    viewer.sync()
    time.sleep(0.02)


Root dir: C:\Users\rad\itmo\new_roms
Models dir: C:\Users\rad\itmo\new_roms\project\models
Clock scene XML exists: False
XML path: C:\Users\rad\itmo\new_roms\project\models\hand_manipulate_clock.xml


ValueError: mjParseXML: could not open file 'C:\Users\rad\itmo\new_roms\project\models\hand_manipulate_clock.xml'